## **Character-Aware Neural Language Models**<br><br>

### **Contribution**<br>
character level의 embedding을 통해 word-level language model의 성능을 향상하는 방법을 제시했다<br><br>
네트워크 자체는 당시 보편적인 것을 사용했으나, 입력 데이터 임베딩을 char-level로 사용함으로써 word-level embedding이 정말 필요한 것인가?라는 의문을 제기한다.영어, 체코어, 독일어, 스페인어, 불어, 러시아어, 아랍어 데이터셋으로 실험을 진행했다. 형태소가 풍부한 언어일수록 성능 차이가 word-level 대비 좋게 나왔다. 이러한 장점들이 있지만 char-level 자체가 efficienty - time의 tradeoff가 있다<br><br><br><br>
### **Abstract**
오로지 char-level inputs에만 의존하는 심플 NLM을 설명할 것이다. 다음 단어에 대한 predictions는 여전히 word-level에서 이루어진다. 이 모델은 characters에 대해서 CNN과 Highway Network를 사용하며 output은 LSTM으로 들어간다.<br><br>
60%더 적은 파라미터에도 불구하고 English Penn Treebank (PTB) 데이터셋에서 현존하는 SOTA(2015년 기준)에 준하는 성능을 보였다.형태소가 다양한 언어들에서 이 모델은 word-level / morpheme-level (단어 레벨 / 형태소 레벨) LSTM 베이스라인을 넘어선다.<br><br>
여전히 더 적은 파라미터들에도 불구하고, 논문 결과는 많은 언어들에서 Language Modeling (언어 모델링)에 character inputs가 sufficient 하다는 걸 시사한다.<br><br><br><br>
이 논문 모델의 태스크는 next word prediction이다<br><br>
![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbAK3bn%2FbtqFfuNXW6B%2Fedf9seGCNhhpQVwKa7Q8rK%2Fimg.png)<br><br><br><br><br>

## **요약본**<br><br>
- epoch는 아랍어가 아니라면 25번, 아랍어는 30번<br>
- backpropagation은 35 time steps만 진행 (코드로 보면 sentence 길이가 35가 넘어가면 자름)<br>
- learning rate는 1로 시작해서 0.5 weight decay<br><br>
- 모델의 파라미터들은 [-0.05, 0.05]의 uniform distribution으로 random initialization<br>
- dropout = 0.5 (다만 Highway network -> LSTM으로 최초로 갔을 땐 dropout 적용을 하지 않는다)<br>
- Gradient norm = 5 (5가 넘어가면 다시 5로 renormalization)<br><br>
 - Hierarchical Softmax는 데이터 사이즈를 키웠을 때 DATA-L에만 적용해주었다. DATA-S 상태에서는 효과가 없었다<br><br><br>
### 1. preprocessing 단계<br>
데이터셋은 언어별로 다르며, 영어의 경우 PTB 데이터셋 (Mikolov가 전처리한 버전, voab size는 10K) 사용<br>
3 종류의 embedding matrices를 사용<br>
1) character-level embeddding matrix (input으로 들어갈 땐 word 형태로 들어감, 이 논문의 contribution)<br>
2) word-level embedding matrix (비교군)<br>
3) morpheme matrix (prefix(접두사) + stem(어간) + suffix(접미사)) + word-level embedding matrix<br><br>
여기서 3)번의 morpheme matrix는 morpheme을 얻기 위해서 Morfessor Cat-MAP(Creutz and Lagus 3007, unsupervised morphological tagger)을 썼다고 함<br><br><br><br>
### 2. CharCNN 단계<br>
1) CNN 코드상의 kernel width w = [1.2.3.4.5.6.7] (large model 기준, small model은 1~6까지만)이고, height는 15-dim이다. hidden 수는 [50, 100, 150, 200, 200, 200, 200]<br>
2) 각 convolution마다 max값을 취하고, 그렇게 한 번 stride하고 tanh를 적용시켜주고 나면, 첫 번째 필터에 대한 max값들의 모음인 vector 하나가 나온다. 이 vector의 max값을 다시 취한다<br>
3) max값끼리 concatenate를 한다. 이렇게 나온 벡터가 뒤의 Highway Network의 input이 된다<br><br><br><br>
### 3. Highway Network 단계<br>
다른 논문에서는 x였던 것을, 여기서는 CharCNN의 output값인 y로 넣어준다.<br><br>
1) y를 받아서 affine transformation W_(T) * y + b_(T) 연산을 해준다. (임베딩 행렬 x 인풋 + 바이어스)<br>
2) 위의 결과값에 sigmoid를 씌운다. 이 값이 transform gate t가 된다<br>
3) 다시 2)번의 결과값과 ReLU(W__(H) * y + b_(H))를 element-wise product를 해준다.<br>
4) 다음으로 (1-t) 즉, 숫자 1에서 2)의 결과값을 빼준다. 이게 carry gate이다<br>
5) 4)번의 carry gate와 y를 곱해준다<br>
6) 마지막으로 3)의 값과 5)의 값을 더해서 LSTM으로 보낸다<br><br>
- 위의 W_(T)와 W_(H)는 모두 연산의 편리성을 위해 square matrix (정방행렬)이며, 파이토치 코드로는 nn.Linear(dim, dim)이 된다.<br><br>
- Highway Network의 bias들은 -2로 들어간다.<br>
- 레이어의 개수는 2개가 좋다. 그 이상으로 늘려도 더 효과는 없음<br><br>
- 이 논문에서 Highway Network는 semantic features를 아는 것 같다고 함(character-level input을 더 의미있게 만들어줌. 왜냐하면 여기를 통과하기 전 상태에서는 단어의 외부적인 모습, --you, your, young 등등을 인식한 모습을 보여주는데, Highway를 통과한 이후에는 you, we같은 semantic한 것들도 잡아내기 때문<br><br>
이는 통과하기 전과 통과한 후의 Nearest Neightbor를 적용해서 체크했다.<br><br>
- Ax + b 형태의 수식 자체가 affine transformation이다. Ax는 그냥 선형이고, b가 있어야 affine transformation<br><br>


